### Importing Libraries & Loading the dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

df = pd.read_csv('../data/bank_data_C.csv', parse_dates=True)


### General Overview

In [2]:
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [3]:
df.tail()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
1041609,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0
1041610,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0
1041611,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0
1041612,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0
1041613,T1048567,C8337524,5/3/84,M,PUNE,75734.42,18/9/16,181222,1166.0


In [4]:
df.shape

(1041614, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041614 entries, 0 to 1041613
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1041614 non-null  object 
 1   CustomerID               1041614 non-null  object 
 2   CustomerDOB              1041614 non-null  object 
 3   CustGender               1041614 non-null  object 
 4   CustLocation             1041614 non-null  object 
 5   CustAccountBalance       1041614 non-null  float64
 6   TransactionDate          1041614 non-null  object 
 7   TransactionTime          1041614 non-null  int64  
 8   TransactionAmount (INR)  1041614 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 71.5+ MB


At a glance we can see that there are no missing values.

#### Checking duplicates

In [6]:
df = df.drop('TransactionID', axis=1)

In [7]:
df.duplicated().sum()

np.int64(0)

There are no missing values, even after dropping the unique identifing column.

### Correcting datatypes

In [8]:
df.dtypes

CustomerID                  object
CustomerDOB                 object
CustGender                  object
CustLocation                object
CustAccountBalance         float64
TransactionDate             object
TransactionTime              int64
TransactionAmount (INR)    float64
dtype: object

In [9]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

C:\Users\godfr\AppData\Local\Temp\ipykernel_8284\617845747.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])


In [23]:
# Convert to datetime, assuming day/month/year format
df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'])

In [11]:
df.iloc[16]

CustomerID                            C1376215
CustomerDOB                1800-01-01 00:00:00
CustGender                                   M
CustLocation                            MUMBAI
CustAccountBalance                    77495.15
TransactionDate            2016-01-08 00:00:00
TransactionTime                         124727
TransactionAmount (INR)                1423.11
Name: 16, dtype: object

In [12]:
df['CustomerDOB'].describe()

count                          1041614
mean     1985-07-03 08:17:32.308436800
min                1800-01-01 00:00:00
25%                1983-12-08 00:00:00
50%                1988-12-13 00:00:00
75%                1992-08-22 00:00:00
max                2074-12-31 00:00:00
Name: CustomerDOB, dtype: object

There appears to be a few invalid dates in the CustomerDOB column. The minimum date is 01/01/1800 which make some of your customers over 200 years olds.

We also have a maximum date of 31/12/2074 in our dataset when the last transaction date is 09/12/2016. Thus some of our customers have transacted with us before their were born. This is not logically so we are going to investigate and handle these inconsisent dates before proceeding.

we would first start by filtering out the dates lower than the 1st quartile('1983-01-01'). This date was choosen becasue it is a valid date and it can be used as a starting point to trace the logical dates.

In [13]:
# filter out all dates lower than '1983-01-01'
df[df['CustomerDOB'] < df['CustomerDOB'].quantile(0.25)]

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
7,C1220223,1982-01-27,M,MUMBAI,95075.54,2016-02-08,170537,148.00
10,C5430833,1982-07-22,M,MOHALI,48429.49,2016-02-08,204133,259.00
12,C6339347,1978-06-13,M,AHMEDABAD,32274.78,2016-02-08,203834,12300.00
14,C7917151,1978-03-24,M,PUNE,10100.84,2016-01-08,82253,338.00
16,C1376215,1800-01-01,M,MUMBAI,77495.15,2016-01-08,124727,1423.11
...,...,...,...,...,...,...,...,...
1041595,C2153578,1981-09-03,M,BANGALORE,11247.35,2016-09-18,184225,9363.00
1041600,C2210842,1981-07-22,M,DELHI,28498.26,2016-09-18,185151,806.00
1041602,C2122724,1978-03-20,M,FARIDABAD,436598.03,2016-09-18,185337,5259.00
1041607,C5028150,1980-05-24,M,PUNE,464.87,2016-09-18,184632,3000.00


We can see from glancing through the that we have customers were born in the year '1978' in the data. The year '1970' would therefore be used as our next investigative point.

In [14]:
invalid_dates = df[df['CustomerDOB'] < pd.to_datetime('1970-01-01')]
invalid_dates

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
16,C1376215,1800-01-01,M,MUMBAI,77495.15,2016-01-08,124727,1423.11
22,C7018081,1800-01-01,M,WAYS PANCHKUVA AHMEDABAD,143.07,2016-03-08,104718,110.00
28,C7935438,1800-01-01,M,MOHALI,969436.12,2016-03-08,183634,36.90
34,C1211387,1800-01-01,M,AHMEDABAD,11791.25,2016-03-08,125915,1075.00
150,C3334638,1800-01-01,M,PUNE,700.00,2016-09-08,24912,1500.00
...,...,...,...,...,...,...,...,...
1041491,C7261226,1800-01-01,M,BHILAI,1393.33,2016-09-18,102427,200.00
1041500,C3813481,1800-01-01,M,NEW DELHI,984605.24,2016-09-18,84644,219.00
1041578,C6937679,1800-01-01,M,NEW DELHI,50698.90,2016-09-18,184319,598.86
1041580,C7431884,1800-01-01,M,MANDIR DHANBAD,53771.64,2016-09-18,184049,520.00


The above dataset now has '1800-01-01' in this customerDOB column. Due to the fact that it is a single date that runs throughout the entire dataset, it maybe a system error.

Let us find out the number of rows and the proportion of our data that has these erroroeus dates.

In [15]:
# print(display(invalid_dates['CustomerDOB'].value_counts()))

print(len(invalid_dates)/len(df))

print(len(invalid_dates)/len(df)*100)

0.05404305241673019
5.404305241673019


We have a total of 56,292 rows in our dataset containing these erroroeus dates. This amounts to 5.4% of the dataset.

We would drop these rows from our dataset because it would be impossible to correct them.

In [16]:
df = df.drop(invalid_dates.index, axis=0)

In [17]:
# Format back to string with 4-digit year
df['CustomerDOB'] = df['CustomerDOB'].dt.strftime('%d/%m/%Y')

def convert_year(date_str):
  # split the date into 3 items and map each item to the variables
  day, month, year = map(str, date_str.split('/'))
  # Getting the last 2 digits from the year
  last_2 = int(year[2:])
  # if the last to digits is less than 20 than the person was born with 2000 - 2020
  if last_2 <= 20:
    year = 2000 + last_2
  # if the last to digits is greater than 20 than the person was born before 2000
  else:
    year = 1900 + last_2
  # return the date as a single string to represent the date
  return f"{day}/{month}/{year}"

df['CustomerDOB'] = df['CustomerDOB'].apply(convert_year)

# Convert the date as string to datetime object
df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'], format="%d/%m/%Y")

df.head()

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0
1,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,141858,27999.0
2,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,142712,459.0
3,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.0
4,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5


In [18]:
df.shape

(985322, 8)

In [19]:
today = pd.to_datetime('today')
today

Timestamp('2025-10-11 07:38:39.684884')

In [20]:
# How old was each customer at the point of their last tranasction?
today = pd.to_datetime('today')
df['CustomerAge'] = (today - df['CustomerDOB']).dt.days // 365
df.head()

,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge
0,C5841053,1994-10-01,F,JAMSHEDPUR,17819.05,2016-02-08,143207,25.0,31
1,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-02-08,141858,27999.0,68
2,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-02-08,142712,459.0,28
3,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-02-08,142714,2060.0,52
4,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-02-08,181156,1762.5,37


In [21]:
df['CustomerAge'].describe()

count    985322.000000
mean         39.848611
std           8.764251
min           5.000000
25%          34.000000
50%          38.000000
75%          43.000000
max         104.000000
Name: CustomerAge, dtype: float64

We would now check for customer date of birth that results in the customer being less than 18 years.

We would perform this operation based on the assumption that only customers older than 18 years are allowed to open an account and transact with the company.

In [22]:
# Selecting customers with age greater than 18
df = df[df['CustomerAge'] > 18]